<a href="https://colab.research.google.com/github/jaegertros/cov-d-19-data-notebooks/blob/main/scripts/notebooks/who.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate CSVs from COVID-19 WHO data

## Dependencies

In [ ]:
import pandas as pd
import os
from datetime import datetime

INPUT_PATH = 'input/'
OUTPUT_PATH = 'public/data/'

POPULATION_CSV_PATH = os.path.join(INPUT_PATH, 'un', 'population_thousands.csv')
REGIONS_CSV_PATH = os.path.join(INPUT_PATH, 'regions.csv')
DEATHS_CSV_PATH = os.path.join(INPUT_PATH, 'deaths.csv')
CASES_CSV_PATH = os.path.join(INPUT_PATH, 'cases.csv')

In [ ]:
!mkdir -p $OUTPUT_PATH

## Load the data

Population figures

In [ ]:
df_pop = pd.read_csv(POPULATION_CSV_PATH)

In [ ]:
# WHO includes counts for Kosovo in Serbia
# https://www.who.int/countries/srb/en/
df_pop['entity'] = df_pop['entity'].replace({ 'Serbia (including Kosovo)': 'Serbia' })

In [ ]:
df_pop['population'] = df_pop['population_thousands'] * 1000

In [ ]:
def find_closest_year(df, year=2020):
    df = df.copy()
    df['year'] = df['year'].sort_values(ascending=True)
    return df.loc[df['year'].map(lambda x: abs(x - 2020)).idxmin()]

In [ ]:
current_pop = pd.DataFrame([
    find_closest_year(df, 2020)
    for loc, df in df_pop.groupby('entity')
]) \
.dropna() \
.drop(columns=['population_thousands']) \
.rename(columns={'entity': 'location'})

WHO regions & OWID names

In [ ]:
df_regions = pd.read_csv(REGIONS_CSV_PATH)

In [ ]:
df_regions = df_regions.merge(
    current_pop[['location', 'population']],
    how='left',
    left_on='OWID Country Name',
    right_on='location'
).drop(columns=['location'])
# Convert population figures to ints
df_regions['population'] = df_regions['population'].round().astype('Int64')
# Add a more descriptive header
df_regions = df_regions.rename(columns={'population': 'Population (UN 2020 projection)'})

Transform the "wide" format into "long" format, which is easier to work with.

In [ ]:
def melt_csv(df, var_name):
    return df.melt(
        id_vars=df.columns[0], 
        value_vars=df.columns[1:], 
        var_name='location', 
        value_name=var_name
    ).dropna()

In [ ]:
df_deaths = melt_csv(pd.read_csv(DEATHS_CSV_PATH, header=1).rename(columns={ 'Date': 'date' }), 'total_deaths')
df_cases = melt_csv(pd.read_csv(CASES_CSV_PATH, header=1).rename(columns={ 'Date': 'date' }), 'total_cases')

Convert all numbers from floating point to integers:

In [ ]:
df_deaths['total_deaths'] = df_deaths['total_deaths'].astype('Int64')
df_cases['total_cases'] = df_cases['total_cases'].astype('Int64')

## Calculations

Join cases & deaths into one dataframe

In [ ]:
df_merged = df_cases.merge(
    df_deaths, 
    how='outer', 
    left_on=['date', 'location'], 
    right_on=['date', 'location']
).sort_values(by=['location', 'date'])

Standardize names to OWID names

In [ ]:
df_regions_merged = df_regions.merge(
    df_merged[['location']].drop_duplicates(),
    how="outer",
    left_on="WHO Country Name",
    right_on="location"
)

In [ ]:
df_regions_merged[df_regions_merged['OWID Country Name'].isnull()]

,WHO Country Name,WHO Region,OWID Country Name,Population (UN 2020 projection),location


In [ ]:
assert(df_regions_merged['OWID Country Name'].isnull().any() == False)

In [ ]:
who_name_replace_map = { r['WHO Country Name']: r['OWID Country Name'] for r in df_regions_merged.to_dict('records') }

In [ ]:
df_merged['location'] = df_merged['location'].replace(who_name_replace_map)

Calculate daily cases & deaths

In [ ]:
# Convert to Int64 to handle <NA>
df_merged['new_cases'] = df_merged.groupby('location')['total_cases'].diff().astype('Int64')
df_merged['new_deaths'] = df_merged.groupby('location')['total_deaths'].diff().astype('Int64')

Create a `World` aggregate

In [ ]:
df_global = df_merged.groupby('date').sum().reset_index()
df_global['location'] = 'World'

In [ ]:
df_merged = pd.concat([df_merged, df_global], sort=True)

Calculate per population variables

In [ ]:
df_merged_pop = df_merged.merge(
    current_pop,
    how='left',
    on='location'
)

In [ ]:
# These entities do not exist in the UN population dataset but do in the WHO data
df_merged_pop[df_merged_pop['population'].isnull()]['location'].unique()

array(['Andorra', 'Cayman Islands', 'Faeroe Islands', 'Gibraltar',
       'Guadeloupe', 'Guernsey', 'Vatican', 'International', 'Jersey',
       'Liechtenstein', 'Monaco', 'Saint Barthlemy',
       'Saint Martin (French part)', 'San Marino'], dtype=object)

In [ ]:
df_merged_pop['total_cases_per_million'] = df_merged_pop['total_cases'] / (df_merged_pop['population'] / 1e6)
df_merged_pop['total_deaths_per_million'] = df_merged_pop['total_deaths'] / (df_merged_pop['population'] / 1e6)
df_merged_pop['new_cases_per_million'] = df_merged_pop['new_cases'] / (df_merged_pop['population'] / 1e6)
df_merged_pop['new_deaths_per_million'] = df_merged_pop['new_deaths'] / (df_merged_pop['population'] / 1e6)

In [ ]:
df_merged = df_merged_pop.drop(columns=['year', 'population'])

Calculate days since 100th case

In [ ]:
THRESHOLD = 100
DAYS_SINCE_COL_NAME = 'days_since_%sth_case' % THRESHOLD
DAYS_SINCE_COL_NAME_POSITIVE = 'days_since_%sth_case_positive' % THRESHOLD

In [ ]:
def get_date_of_nth_case(df, nth):
    try:
        df_gt_nth = df[df['total_cases'] >= nth]
        earliest = df.loc[pd.to_datetime(df_gt_nth['date']).idxmin()]
        return earliest['date']
    except:
        return None

In [ ]:
date_of_nth_case = pd.DataFrame([
    (loc, get_date_of_nth_case(df, THRESHOLD)) 
    for loc, df in df_merged.groupby('location')
], columns=['location', 'date_of_nth_case']).dropna()

In [ ]:
def inject_days_since(df, ref_date):
    df = df[['date', 'location']].copy()
    df[DAYS_SINCE_COL_NAME] = df['date'].map(lambda date: (pd.to_datetime(date) - pd.to_datetime(ref_date)).days)
    return df

In [ ]:
df_grouped = df_merged.groupby('location')
df_days_since_nth_case = pd.concat([
    inject_days_since(df_grouped.get_group(row['location']), row['date_of_nth_case']) 
    for _, row in date_of_nth_case.iterrows()
])

In [ ]:
df_merged = df_merged.merge(
    df_days_since_nth_case,
    how='outer',
    on=['date', 'location'],
)
df_merged[DAYS_SINCE_COL_NAME] = df_merged[DAYS_SINCE_COL_NAME].astype('Int64')
df_merged[DAYS_SINCE_COL_NAME_POSITIVE] = df_merged[DAYS_SINCE_COL_NAME] \
    .map(lambda x: x if (pd.notna(x) and x >= 0) else None).astype('Int64')

Calculate doubling rates

In [ ]:
def get_days_to_double(df, col_name):
    try:
        # verbose because being very careful not to modify original data with dates
        latest = df.loc[pd.to_datetime(df['date']).idxmax()]
        df_lt_half = df[df[col_name] <= (latest[col_name] / 2)]
        half = df_lt_half.loc[pd.to_datetime(df_lt_half['date']).idxmax()]
        return (pd.to_datetime(latest['date']) - pd.to_datetime(half['date'])).days
    except:
        return None

In [ ]:
days_to_double_cases = pd.DataFrame([
    (loc, get_days_to_double(df, 'total_cases')) 
    for loc, df in df_merged.groupby('location')
], columns=['location', 'days_to_double_cases'])
days_to_double_cases['days_to_double_cases'] = days_to_double_cases['days_to_double_cases'].astype('Int64')

### Grapher data extract

In [ ]:
df_grapher = df_merged.copy()
df_grapher['date'] = pd.to_datetime(df_grapher['date']).map(lambda date: (date - datetime(2020, 1, 21)).days)
df_grapher = df_grapher[[
    'location', 'date', 
    'new_cases', 'new_deaths', 
    'total_cases', 'total_deaths',
    'new_cases_per_million', 'new_deaths_per_million', 
    'total_cases_per_million', 'total_deaths_per_million',
    DAYS_SINCE_COL_NAME, DAYS_SINCE_COL_NAME_POSITIVE]] \
    .rename(columns={
        'location': 'country',
        'date': 'year',
        'new_cases': 'Daily new confirmed cases of COVID-19',
        'new_deaths': 'Daily new confirmed deaths due to COVID-19',
        'total_cases': 'Total confirmed cases of COVID-19', 
        'total_deaths': 'Total confirmed deaths due to COVID-19',
        'new_cases_per_million': 'Daily new confirmed cases of COVID-19 per million people',
        'new_deaths_per_million': 'Daily new confirmed deaths due to COVID-19 per million people',
        'total_cases_per_million': 'Total confirmed cases of COVID-19 per million people', 
        'total_deaths_per_million': 'Total confirmed deaths due to COVID-19 per million people',
        DAYS_SINCE_COL_NAME: 'Days since the total confirmed cases of COVID-19 reached %s' % THRESHOLD,
        DAYS_SINCE_COL_NAME_POSITIVE: 'Days since the total confirmed cases of COVID-19 reached %s (positive only)' % THRESHOLD,
    
    })

## Inspect the results

In [ ]:
# df_merged

In [ ]:
# df_merged[df_merged['location'] == 'World']

## Write output files

In [ ]:
# Should keep these append-only in case someone external depends on the order
df_merged[[
    'date', 'location',
    'new_cases', 'new_deaths', 
    'total_cases', 'total_deaths']] \
    .to_csv(os.path.join(OUTPUT_PATH, 'full_data.csv'), index=False)

In [ ]:
for col_name in [
    'total_cases', 'total_deaths', 
    'new_cases', 'new_deaths',
    'total_cases_per_million', 'total_deaths_per_million', 
    'new_cases_per_million', 'new_deaths_per_million']:
    df_pivot = df_merged.pivot(index='date', columns='location', values=col_name)
    # move World to first column
    cols = df_pivot.columns.tolist()
    cols.insert(0, cols.pop(cols.index('World')))
    df_pivot[cols].to_csv(os.path.join(OUTPUT_PATH, '%s.csv' % col_name))

In [ ]:
days_to_double_cases.to_csv(os.path.join(OUTPUT_PATH, 'days_to_double_cases.csv'), index=False)

In [ ]:
df_regions.to_csv(os.path.join(OUTPUT_PATH, 'regions.csv'), index=False)

In [ ]:
df_grapher.to_csv(os.path.join(OUTPUT_PATH, 'grapher.csv'), index=False)